# <span style="color:Maroon">Data Preparation -- Part 1

<span style="color:Green">__Overview:__ Create dataset at county level. The columns store the values of Mortality rate (mean, upper and lower bounds) for different causes.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import os as os
import xlrd

In [2]:
os.chdir("..")
cwd = os.getcwd()

In [3]:
# Define the directories based on type of data
country_category_year = "Data\\country_category_year"
labels = "Data\\labels"
state_category_year = "Data\\state_category_year"
states_gender_year_cause = "Data\\states_gender_year_cause"

In [4]:
# define the filenames based on type of data
file_coun = "IHME_USA_COUNTY_MORTALITY_RATES_1980_2014_NATIONAL_Y2016M12D13.xlsx"
file_label = "IHME_USA_COUNTY_MORTALITY_RATES_1980_2014_CODEBOOK_Y2017M05D19.csv"
file_st_ct_yr1 = "IHME_USA_COUNTY_MORTALITY_RATES_1980_2014_"
file_st_ct_yr2 = "_Y2016M12D13"
file_st_gn_yr_cs1 = "IHME_USA_COUNTY_MORTALITY_RATES_1980_2014_"
file_st_gn_yr_cs2 = "_Y2017M05D19"

In [5]:
# List with state names
states = ["ALABAMA", "ALASKA", "ARIZONA", "ARKANSAS", "CALIFORNIA", "COLORADO", "CONNECTICUT", "DELAWARE", 
          "DISTRICT_OF_COLOMBIA", "FLORIDA", "GEORGIA", "HAWAII", "IDAHO", "ILLINOIS", "INDIANA", "IOWA", "KANSAS", 
          "KENTUCKY", "LOUISIANA", "MAINE", "MARYLAND", "MASSACHUSETTS", "MICHIGAN", "MINNESOTA", "MISSISSIPPI",
          "MISSOURI", "MONTANA", "NEBRASKA", "NEVADA", "NEW_HAMPSHIRE", "NEW_JERSEY", "NEW_MEXICO", "NEW_YORK",
          "NORTH_CAROLINA", "NORTH_DAKOTA", "OHIO", "OKLAHOMA", "OREGON", "PENNSYLVANIA", "RHODE_ISLAND", 
          "SOUTH_CAROLINA", "SOUTH_DAKOTA", "TENNESSEE", "TEXAS", "UTAH", "VERMONT", "VIRGINIA", "WASHINGTON", 
          "WEST_VIRGINIA", "WISCONSIN", "WYOMING"]

In [6]:
# List with different categories
categories = ["HIV-AIDS & tuberculosis", "Diarrhea, lower respiratory", "Neglected tropical diseases",
              "Maternal disorders", "Neonatal disorders", "Nutritional deficiencies", "Other communicable, maternal, ",
              "Neoplasms", "Cardiovascular diseases", "Chronic respiratory diseases", "Cirrhosis & other chronic ",
              "Digestive diseases", "Neurological disorders", "Mental & substance use ", "Diabetes, urogenital, blood",
              "Musculoskeletal disorders", "Other non-communicable ", "Transport injuries", "Unintentional injuries",
              "Self-harm & interpersonal ", "Forces of nature, war, & legal "]

In [7]:
# Dictionary mapping excel tab name to acronym
category_dict = {"HVT":"HIV-AIDS & tuberculosis",
                 "DLR":"Diarrhea, lower respiratory",
                 "NTD":"Neglected tropical diseases",
                 "MDA":"Maternal disorders",
                 "NDA":"Neonatal disorders",
                 "NEA":"Nutritional deficiencies",
                 "OTA":"Other communicable, maternal, ",
                 "NOA":"Neoplasms",
                 "CDA":"Cardiovascular diseases",
                 "CRE":"Chronic respiratory diseases",
                 "COC":"Cirrhosis & other chronic ",
                 "DDA":"Digestive diseases", 
                 "NIA":"Neurological disorders",
                 "MSU":"Mental & substance use ",
                 "DUB":"Diabetes, urogenital, blood",
                 "MIA":"Musculoskeletal disorders",
                 "ONC":"Other non-communicable ",
                 "TIA":"Transport injuries",
                 "UIA":"Unintentional injuries",
                 "SHI":"Self-harm & interpersonal ",
                 "FON":"Forces of nature, war, & legal "}

In [8]:
category_key = ["HVT","DLR","NTD","MDA","NDA","NEA","OTA","NOA","CDA","CRE","COC","DDA","NIA","MSU","DUB","MIA","ONC","TIA"
               ,"UIA","SHI","FON"]

In [9]:
# Check if the list is unique (no two similar names)
len(category_key) == len(list(dict.fromkeys(category_key)))

True

In [10]:
country = "UNITED_STATES"

## <span style="color:Maroon">Read the label file

In [11]:
# Label file
os.chdir(f'{cwd}\\{labels}')
labels = pd.read_csv(file_label)
labels.head()

,Variable:,location_id,location_name,FIPS,cause_id,cause_name,sex_id,sex,year_id,mx,upper,lower
0,Label:,Location ID,Location name,FIPS,Cause ID,Cause name,Sex ID,Sex name,Year ID,Mortality rate,95% Uncertainty Interval - Upper Bound,95% Uncertainty Interval - Lower Bound
1,Value coding:,102,United States,NaN,294,All causes,1,Male,1980,NaN,NaN,NaN
2,NaN,523,Alabama,1,295,"Communicable, maternal, neonatal, and nutritio...",2,Female,1981,NaN,NaN,NaN
3,NaN,524,Alaska,2,296,HIV/AIDS and tuberculosis,3,Both,1982,NaN,NaN,NaN
4,NaN,525,Arizona,4,301,"Diarrhea, lower respiratory, and other common ...",NaN,NaN,1983,NaN,NaN,NaN


## <span style="color:Maroon">Read data at state level with different causes

In [12]:
def SplitCol_toRows(df, column_index):
    columns = df.columns
    data = df.copy()
    for i in range(0,len(column_index)):
        column_name = columns[column_index[i]]
        var1 = column_name + "_VAL"
        var2 = column_name + "_LB"
        var3 = column_name + "_UB"
        data[[var1, "tmp"]] = data[column_name].str.split("(", expand=True)
        data[[var2, var3]] = data["tmp"].str.split(",", expand=True)
        data[var3] = data[var2].str.replace(")",'')
        data[var1] = data[var1].astype(float)
        data[var2] = data[var2].astype(float)
        data[var3] = data[var3].astype(float)
        data = data.drop(["tmp",column_name], axis=1)
    return data

In [13]:
def perct_change(df, column_name1,column_name2,key):
    df[key+"_PC1980_2010"] = (df[column_name1]-df[column_name2])/df[column_name2]
    return df

In [14]:
# Data at country level
os.chdir(f'{cwd}\\{country_category_year}')

In [15]:
combined_data = pd.DataFrame()
for i in range(0, len(category_key)):
    sheetname = category_dict[category_key[i]]
    data = pd.read_excel(file_coun, sheet_name=sheetname,header=1,skipfooter=2, engine='openpyxl')
    list1 = ["MR1980","MR1985","MR1990","MR1995","MR2000","MR2005","MR2010"]
    list2 = ["MR2014","PC1980_2014"]
    list1 = [category_key[i]+"_"+sub for sub in list1]
    data.columns =list(data.columns[0:2])+list1+list2
    data = data.drop(list2,axis=1)
    column_index = [2,3,4,5,6,7,8]
    data = SplitCol_toRows(data,column_index)
    data = perct_change(data, category_key[i]+"_"+"MR2010_VAL",category_key[i]+"_"+"MR2005_VAL", category_key[i])
    data = data.drop("Location",axis=1)
    data["FIPS"] = data["FIPS"].fillna(0).astype(int)
    data = data.set_index("FIPS")
    combined_data = combined_data.merge(data,how="outer",left_index=True,right_index=True)

In [16]:
combined_data.head()

,HVT_MR1980_VAL,HVT_MR1980_LB,HVT_MR1980_UB,HVT_MR1985_VAL,HVT_MR1985_LB,HVT_MR1985_UB,HVT_MR1990_VAL,HVT_MR1990_LB,HVT_MR1990_UB,HVT_MR1995_VAL,...,FON_MR2000_VAL,FON_MR2000_LB,FON_MR2000_UB,FON_MR2005_VAL,FON_MR2005_LB,FON_MR2005_UB,FON_MR2010_VAL,FON_MR2010_LB,FON_MR2010_UB,FON_PC1980_2010
FIPS,,,,,,,,,,,,,,,,,,,,,
0,1.52,1.44,1.44,3.16,3.11,3.11,11.45,11.34,11.34,16.61,...,0.20,0.11,0.11,1.20,0.37,0.37,0.23,0.13,0.13,-0.808333
1,1.46,1.33,1.33,2.15,2.03,2.03,8.03,7.79,7.79,14.10,...,0.47,0.25,0.25,1.81,0.42,0.42,0.39,0.19,0.19,-0.784530
1001,0.95,0.68,0.68,1.44,1.12,1.12,6.57,5.49,5.49,10.75,...,0.23,0.03,0.03,1.43,0.11,0.11,0.31,0.04,0.04,-0.783217
1003,0.84,0.63,0.63,1.44,1.18,1.18,6.51,5.71,5.71,11.18,...,0.19,0.03,0.03,1.07,0.08,0.08,0.20,0.03,0.03,-0.813084
1005,1.11,0.82,0.82,1.81,1.43,1.43,7.86,6.66,6.66,15.65,...,0.30,0.04,0.04,1.70,0.14,0.14,0.34,0.04,0.04,-0.800000


In [18]:
combined_data.to_csv("..//Model data//Mortality_Category_County_5yr_Data.csv")